# Course: ICS 574
# Prject: Alchemists Project - EDA
- Date: May 1st 2024
- Project Members
    - AHMED DHAFER ALQARNI, ID: 201453160
    - WALEED ABDULLAH ALFAIFI, ID: 201640920
    - ALGHAMDI, BANDAR, ID: 202206560

In [6]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor



In [8]:
# Create a Spark session
spark = SparkSession.builder.appName("Decision Tree Classifier Example 3").getOrCreate()




24/03/27 10:10:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
# Load the datasets as pandas DataFrames
stocks_df = pd.read_csv("./datasets/Tadawul_stcks.csv")
brent_df = pd.read_csv("./datasets/BrentOilPrices.csv")
gold_df = pd.read_csv("./datasets/Gold_Daily.csv")

# Preprocess the stock data
stocks_specific_df = stocks_df[(stocks_df['sectoer'] == 'Energy') | (stocks_df['sectoer'] == 'Materials')]
stocks_specific_df.rename(columns={'date': 'Date'}, inplace=True)
stocks_specific_df.rename(columns={'close': 'Stock_Price'}, inplace=True)
stocks_specific_df['Date'] = pd.to_datetime(stocks_specific_df['Date'])
brent_df['Date'] = pd.to_datetime(brent_df['Date'])
gold_df['Date'] = pd.to_datetime(gold_df['Date'])
brent_df.rename(columns={'Price': 'Brent_Price'}, inplace=True)
gold_df.rename(columns={'Price': 'Gold_Price'}, inplace=True)
stocks_specific_df.rename(columns={'sectoer': 'Sector'}, inplace=True)
stocks_specific_df = stocks_specific_df[['Date', 'name', 'Stock_Price', 'Sector', 'symbol']]
brent_df = brent_df[['Date', 'Brent_Price']]
gold_df = gold_df[['Date', 'Gold_Price']]
stocks_specific_df.fillna(method='ffill', inplace=True)
stocks_brent_df = pd.merge(stocks_specific_df, brent_df, on='Date', how='inner')
stocks_brent_gold_df = pd.merge(stocks_brent_df, gold_df, on='Date', how='inner')




/var/folders/8r/wc11r95d3lgcc57lr4h_9m0w0000gn/T/ipykernel_24975/4115176948.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_specific_df.rename(columns={'date': 'Date'}, inplace=True)
/var/folders/8r/wc11r95d3lgcc57lr4h_9m0w0000gn/T/ipykernel_24975/4115176948.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_specific_df.rename(columns={'close': 'Stock_Price'}, inplace=True)
/var/folders/8r/wc11r95d3lgcc57lr4h_9m0w0000gn/T/ipykernel_24975/4115176948.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [10]:
# Create a Spark DataFrame
stocks_brent_gold_df_spark = spark.createDataFrame(stocks_brent_gold_df)

In [11]:
# Create a VectorAssembler for the features
assembler = VectorAssembler(inputCols=["Stock_Price", "Brent_Price", "Gold_Price"], outputCol="features")
stocks_brent_gold_df_spark = assembler.transform(stocks_brent_gold_df_spark)

In [12]:
# Train-test split
splits = stocks_brent_gold_df_spark.randomSplit([0.8, 0.2])
train = splits[0]
test = splits[1]



In [13]:
# Define the DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol="features", labelCol="Stock_Price")


In [14]:
# Train the model
model = dt.fit(train)


In [15]:
# Make predictions
predictions = model.transform(test)

In [16]:
# Evaluate the model
evaluator = RegressionEvaluator(labelCol="Stock_Price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE):", rmse)


Root Mean Squared Error (RMSE): 8.042969898488115


In [17]:
# evaluate the model using accuracy
evaluator = RegressionEvaluator(labelCol="Stock_Price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R2:", r2)

# An R-squared (R2) value of 0.946 indicates that approximately 94.6% of the variance in the stock price can be explained by the features (Brent_Price and Gold_Price) in your model.

# Generally, an R2 value close to 1 indicates that the model is able to explain a large proportion of the variance in the data, which is considered good. However, it's important to interpret this value in the context of your specific problem and domain knowledge.

R2: 0.9460308155664667


In [18]:
# Save the model
model.save("decision_tree_regressor_model 2")

In [21]:
# predict the stock price for the next day
predictions = model.transform(stocks_brent_gold_df_spark)
predictions.show(5)



+-------------------+--------------------+-----------+------+------+-----------+----------+--------------------+------------------+
|               Date|                name|Stock_Price|Sector|symbol|Brent_Price|Gold_Price|            features|        prediction|
+-------------------+--------------------+-----------+------+------+-----------+----------+--------------------+------------------+
|2020-03-05 00:00:00|Saudi Arabia Refi...|       34.9|Energy|  2030|      51.29|    1690.5| [34.9,51.29,1690.5]| 34.65715564486664|
|2020-03-05 00:00:00|Saudi Arabian Oil...|       33.0|Energy|  2222|      51.29|    1690.5| [33.0,51.29,1690.5]| 34.65715564486664|
|2020-03-05 00:00:00|Rabigh Refining a...|       14.8|Energy|  2380|      51.29|    1690.5| [14.8,51.29,1690.5]|15.163399172310012|
|2020-03-05 00:00:00|National Shipping...|      31.15|Energy|  4030|      51.29|    1690.5|[31.15,51.29,1690.5]| 30.75689315068493|
|2020-03-05 00:00:00|Aldrees Petroleum...|       60.1|Energy|  4200|      51

In [22]:
# compare the predicted stock price with the actual stock price
predictions.toPandas().to_csv("predictions.csv")


In [23]:
# Stop the Spark session
spark.stop()